## 3.1 identification

In [ ]:
library(Seurat)
library(SeuratData)
library(ggplot2)
library(patchwork)
library(dplyr)
library(psych)
library(pheatmap)

sub.myo.sc <- subset(MuScle.list.Sc, idents = c("Type 2b myofibers","Type I myofibers","Type 2x myofibers","QSCs-1","Type 2a myofibers","Type 2a/x myofibers","QSCs-2","PSCs"))
VlnPlot(sub.myo.sc, features = c("nFeature_RNA", "nCount_RNA", "percent.mt"), ncol = 3)
sub.myo.sc <- NormalizeData(sub.myo.sc, normalization.method = "LogNormalize", scale.factor = 10000)
FindVariableFeatures(sub.myo.sc, selection.method = "vst", nfeatures = 3000)
all.genes <- rownames(sub.myo.sc)
sub.myo.sc <- ScaleData(sub.myo.sc, features = all.genes)
sub.myo.sc <- RunPCA(sub.myo.sc, features = VariableFeatures(object = sub.myo.sc)) 
sub.myo.sc <- FindNeighbors(sub.myo.sc, dims = 1:5)
sub.myo.sc <- FindClusters(sub.myo.sc, resolution = 0.3)
sub.myo.sc <- RunUMAP(sub.myo.sc, dims = 1:5)
DimPlot(sub.myo.sc, reduction = "umap",cols=my36colors)

## 3.2 Count

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

cell_types = ['Type1', 'Type2', 'Type3', 'Type4', 'Type5', 'Type6', 'Type7', 'Type8', 'Type9']
percentages = [32.35, 0.32, 1.55, 0, 21.51, 14.20, 17.94, 6.28, 5.80]  # Sample percentages

num_vars = len(cell_types)

angles = np.linspace(0, 2 * np.pi, num_vars, endpoint=False).tolist()

percentages += percentages[:1]
angles += angles[:1]

fig, ax = plt.subplots(figsize=(6, 6), subplot_kw=dict(polar=True))
ax.fill(angles, percentages, color='red', alpha=0.25)
ax.plot(angles, percentages, color='red', linewidth=2)  # Change color to red

ax.set_theta_offset(np.pi / 2)
ax.set_theta_direction(-1)

ax.set_rscale('log')
ax.set_rticks([10, 20, 40, 60, 80, 100])  # Less radial ticks
ax.set_rlabel_position(0)

ax.set_xticks(angles[:-1])
ax.set_xticklabels(cell_types)

ax.set_title('EDL Cell Type Proportions', y=1.08)
plt.show()

## 3.3 ClusterGVis

In [ ]:
library(ClusterGVis)
library(org.Hs.eg.db)
library(Seurat)
new.cluster.ids <- c("Type 2x myofibers", "Type 2x/2b myofibers", "Type 1 myofibers", "Type 2b myofibers", "Type 1/2a myofibers", "Type 2a/x myofibers","MuSCs", "Type 2a myofibers", "Myoblasts")
names(new.cluster.ids) <- levels(pbmc)
pbmc <- RenameIdents(pbmc, new.cluster.ids)

pbmc.markers.all <- Seurat::FindAllMarkers(pbmc,
                               only.pos = TRUE,
                               min.pct = 0.25,
                               logfc.threshold = 0.25)

indices_to_remove <- grep("ENSSSCG", pbmc.markers.all$gene)
pbmc.markers.all <- pbmc.markers.all[-indices_to_remove, ]

pbmc.markers <- pbmc.markers.all %>%
  dplyr::group_by(cluster) %>%
  dplyr::top_n(n = 20, wt = avg_log2FC)

st.data <- prepareDataFromscRNA(object = pbmc,
                                diffData = pbmc.markers,
                                showAverage = TRUE)
str(st.data)

enrich_KEGG <- enrichCluster(object = st.data,
                        OrgDb = org.Hs.eg.db,
                        type = "KEGG",
                        organism = "hsa",
                        pvalueCutoff = 0.5,
                        topn = 5,
                        seed = 5201314)

markGenes = unique(pbmc.markers$gene)[sample(1:length(unique(pbmc.markers$gene)),40,
                                             replace = F)]

desired_order <- c("MuSCs", "Myoblasts", "Type 1 myofibers", "Type 1/2a myofibers", "Type 2a myofibers", "Type 2a/x myofibers", "Type 2x myofibers", "Type 2x/2b myofibers", "Type 2b myofibers")  # specify your desired order
st.data$long.res$cell_type <- factor(st.data$long.res$cell_type, levels = desired_order)
cluster_order <- c("cluster 7 (20)", "cluster 9 (17)", "cluster 3 (20)", "cluster 5 (12)", "cluster 8 (20)", "cluster 6 (20)", "cluster 1 (20)", "cluster 2 (19)", "cluster 4 (12)")  # Example: update with your actual cluster names and order
st.data$long.res$cluster_name <- factor(st.data$long.res$cluster_name, levels = cluster_order)
visCluster(object = st.data, plot.type = "line")

For the pseudotime inference, RNA velocity analysis, and transcription factor mining sections, we did not generate any new code.